## `sandtool` module

> This program reads and process input data from clicSAND interface, to convert into diferent input formats such as: `*.csv`, `*.xlsx` or datafile `*.txt`.


Mario R. Peralta A.

Mario.Peralta@ucr.ac.cr

*The Electric Power and Energy Research Laboratory (EPERLab)*

### Structure transformation
Module that allows to integrate the input data of clicSAND structure interface in ``otoole`` so that more efficient solvers like ``cbc`` can be used.

Run in command line

```shell
% otoole setup config config.yaml
% otoole setup csv data_csv
```

Set range of years in file `YEAR.csv` in excel then in python same directory run:

```Python
>>> from sandtool import Sand_Interface 
>>> # Call clicSAND interface
>>> sand_data = Sand_Interface(2015,
...                            2070,
...                            config_path="./config.yaml")
>>> input_sand = sand_data.set_input_data("./InputSand.xlsm")
>>> sand_data.write_sand_config_file()

```

In [1]:
from sandtool import Sand_Interface 
# Call clicSAND interface
config_path = "./Uruguay/Trunk/config.yaml"
sand_xlsx = "./Uruguay/Trunk/InputSand.xlsm"

sand_data = Sand_Interface(2015,
                           2070,
                           config_path=config_path)
input_sand = sand_data.set_input_data(sand_xlsx)
sand_data.write_sand_config_file()


Create empty excel template by running in command line:

```shell
% otoole convert csv excel data_csv sandtool.xlsx sand_config.yaml
```

### Populate data

```Python
>>> from sandtool import Otoole_Interface
>>> # Call otoole interface
>>> otoole_data = Otoole_Interface(
...     input_otoole_path="./sandtool.xlsx",
...     config_sand_path="./sand_config.yaml"
... )
>>> input_otoole = otoole_data.populate_template(sand_data)
>>> _ = otoole_data.write_otoole_data(input_otoole)
```

In [2]:
from sandtool import Otoole_Interface
# Call otoole interface
xlsx_path = "./Uruguay/Trunk/sandtool.xlsx"
config_path = "./Uruguay/Trunk/sand_config.yaml"
otoole_data = Otoole_Interface(
    input_otoole_path=xlsx_path,
    config_sand_path=config_path
)
input_otoole = otoole_data.populate_template(sand_data)
_ = otoole_data.write_otoole_data(input_otoole)


Clean up undefined variables and fields (sets & technologies) they depend on.

Now convert excel data into MathProg data:
```shell
% otoole convert excel datafile sandtool.xlsx sandtool.txt sand_config.yaml
```
#### Process a solution from GLPK
Build model and save it as `sandtool_model.glp`, in same command, solve it whose solution it is saved as `sandtool_sol.sol`. After problem is solved in a new command-line generate directory `results-glpk/` with results in csv extention.

```shell
% glpsol -m OSeMOSYS.txt -d sandtool.txt --wglp sandtool_model.glp --write sandtool_sol.sol

% otoole results glpk csv sandtool_sol.sol results-glpk datafile sandtool.txt sand_config.yaml --glpk_model sandtool_model.glp
```

*Note*: Make sure to have `OSeMOSYS.txt` file in the same directory.

#### Process a solution from CBC
Finally process a solution: Build model and solve it with cbc using `otoole` command-line interface. And finally write a directory `results_csv/` with the results converted into ***.csv** extension based on the excel structure (input).


```shell
% glpsol -m OSeMOSYS.txt -d sandtool.txt --wlp sandtool_model.lp --check

% cbc sandtool_model.lp solve -solu sandtool_sol.sol

% otoole results cbc csv sandtool_sol.sol results_csv excel sandtool.xlsx sand_config.yaml
```
#### Visualise RES

To visualise use the excel file, `sandtool.xlsx` to generate the RES as `*.pdf` named: *res.pdf*.

```shell
% otoole viz res excel sandtool.xlsx res.pdf sand_config.yaml
```

It reports the error:

```shall
KeyError: 'TechnologyToStorage'
```

## Some functionality of `sandTool`

In [ ]:
import sandtool as stool

sand_data = stool.Sand_Interface(2015, 2070)
input_data = sand_data.set_input_data()


Load `YAML` config file and filter parameters in `clicSAND` that does not depend on `YEAR` index [y]: **Time indipendent variables**

In [ ]:
non_params_y = sand_data.index_independent_variable(
    set_label="YEAR"
)

print(non_params_y)

['CapacityToActivityUnit', 'DepreciationMethod', 'DiscountRate', 'ModelPeriodEmissionLimit', 'ModelPeriodExogenousEmission', 'OperationalLife', 'TotalTechnologyModelPeriodActivityLowerLimit', 'TotalTechnologyModelPeriodActivityUpperLimit']


Filter all fields but **result** type (due to input data only considers **set** and **param** types) that are in template `config.yaml` but not in `clicSAND` interface and eventually remove them:

In [ ]:
rm_fields = sand_data.non_required_fields()
print(rm_fields)

['Conversionlh', 'DaySplit', 'Conversionld', 'DaysInDayType', 'StorageLevelDayTypeFinish', 'StorageLevelDayTypeStart', 'Conversionls', 'DaysInDayType', 'StorageLevelDayTypeFinish', 'StorageLevelDayTypeStart', 'StorageLevelSeasonStart', 'CapitalCostStorage', 'Conversionld', 'Conversionlh', 'Conversionls', 'DAILYTIMEBRACKET', 'DaysInDayType', 'DaySplit', 'DAYTYPE', 'DiscountRateStorage', 'MinStorageCharge', 'OperationalLifeStorage', 'ResidualStorageCapacity', 'SEASON', 'StorageLevelStart', 'StorageMaxChargeRate', 'StorageMaxDischargeRate', 'TechnologyFromStorage', 'TechnologyToStorage']


Generate new config yaml file based on clicSAND interface variables:


In [ ]:
sand_data.write_sand_config_file()